In [3]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/code/bm_preproc.py

--2020-06-05 09:41:06--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/code/bm_preproc.py
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 143.204.160.108, 143.204.160.55, 143.204.160.98, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|143.204.160.108|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9400 (9.2K) [application/octet-stream]
Saving to: ‘bm_preproc.py’

bm_preproc.py       100%[===================>]   9.18K  --.-KB/s    in 0s      

2020-06-05 09:41:07 (50.9 MB/s) - ‘bm_preproc.py’ saved [9400/9400]



In [4]:
import bm_preproc

In [5]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta

--2020-06-05 09:41:42--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 143.204.160.6, 143.204.160.98, 143.204.160.55, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|143.204.160.6|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/octet-stream]
Saving to: ‘chr1.GRCh38.excerpt.fasta’

chr1.GRCh38.excerpt 100%[===================>] 791.12K  4.19MB/s    in 0.2s    

2020-06-05 09:41:43 (4.19 MB/s) - ‘chr1.GRCh38.excerpt.fasta’ saved [810105/810105]



In [3]:
# read a genome
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

# Naive Matching Algorithm with counting
def naive_with_counts(p, t):
    occurrences = []
    count_alignments = 0
    count_characters = 0
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        count_alignments += 1
        for j in range(len(p)):  # loop over characters
            count_characters += 1
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return (occurrences, count_alignments, count_characters)

# Boyer Moore Matching Algorithm with counting
def boyer_moore_with_counts(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    count_alignments = 0
    count_characters = 0
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        count_alignments += 1
        for j in range(len(p)-1, -1, -1):
            count_characters += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return (occurrences, count_alignments, count_characters)

In [12]:
p = 'word'
t = 'there would have been a time for such a word'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[40] 41 46


In [13]:
p = 'needle'
t = 'needle need noodle needle'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 20 35


In [15]:
from bm_preproc import BoyerMoore

In [16]:
p = 'word'
t = 'there would have been a time for such a word'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[40] 12 15


In [17]:
p = 'needle'
t = 'needle need noodle needle'
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 5 18


In [4]:
chr1 = readGenome("chr1.GRCh38.excerpt.fasta")
pattern = "GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"
occurrences, num_alignments, num_character_comparisons = naive_with_counts(pattern, chr1)
print(occurrences, num_alignments, num_character_comparisons)

[56922] 799954 984143


In [8]:
from bm_preproc import BoyerMoore
chr1 = readGenome("chr1.GRCh38.excerpt.fasta")
pattern = "GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"
HIGHcase_alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ '
p_bm = BoyerMoore(pattern, HIGHcase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(pattern, p_bm, chr1)
print(occurrences, num_alignments, num_character_comparisons)

[56922] 127974 165191


In [36]:
!wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/code/kmer_index.py

--2020-06-05 10:13:01--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/code/kmer_index.py
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.226.186.29, 13.226.186.126, 13.226.186.166, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.226.186.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 971 [application/octet-stream]
Saving to: ‘kmer_index.py’

kmer_index.py       100%[===================>]     971  --.-KB/s    in 0s      

2020-06-05 10:13:01 (27.5 MB/s) - ‘kmer_index.py’ saved [971/971]



In [89]:
# Approximate Matching with Index Matching Algorithm.
import kmer_index
def approximate_match(p, t, n):
    segment_length = int(round(len(p) / (n+1)))
    all_matches = set()
    index_matches = 0
    for i in range(n+1):
        start = i*segment_length
        end = min((i+1)*segment_length, len(p))
        chr_index = kmer_index.Index(t,8)
        matches = chr_index.query(p[start:end])
        index_matches = index_matches + len(matches)
        # Extend matching segments to see if whole p matches
        for m in matches:
            if m < start or m-start+len(p) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start)
    return len(list(all_matches)),index_matches,list(all_matches)

print(approximate_match('GGCGCGGTGGCTCACGCCTGTAAT', chr1,2))


(19, 90, [273669, 681737, 717706, 262042, 635931, 84641, 160162, 724927, 657496, 160729, 56922, 191452, 551134, 747359, 421221, 147558, 364263, 465647, 429299])


In [101]:
def naive_2mm(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        count = 0
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                count += 1
                if count > 2:
                    match = False
                    break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences
print(naive_2mm('GGCGCGGTGGCTCACGCCTGTAAT', chr1))
print('# occurrences: %d' % len(naive_2mm('GGCGCGGTGGCTCACGCCTGTAAT', chr1)))

[56922, 84641, 147558, 160162, 160729, 191452, 262042, 273669, 364263, 421221, 429299, 465647, 551134, 635931, 657496, 681737, 717706, 724927, 747359]
# occurrences: 19


In [90]:
print(approximate_match('GGCGCGGTGGCTCACGCCTGTAAT', chr1,2))

(19, 90, [273669, 681737, 717706, 262042, 635931, 84641, 160162, 724927, 657496, 160729, 56922, 191452, 551134, 747359, 421221, 147558, 364263, 465647, 429299])


In [62]:
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
#             print(self.index[i][0])
#             print(subseq)
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [36]:
ind = SubseqIndex('ATATAT', 3, 2)
print(ind.index)

[('AAA', 0), ('TTT', 1)]


In [39]:
p = 'TTATAT'
print(p[0:])
print(ind.query(p[0:]))

TTATAT
[]


In [40]:
print(p[1:])
print(ind.query(p[1:]))

TATAT
[1]


In [17]:
chr1_subseq_index = SubseqIndex(chr1,8,3)

In [79]:
t = 'to-morrow and to-morrow and to-morrow creeps in this petty pace'
p = 'to-morrow and to-morrow '
subseq_ind = SubseqIndex(t, 8, 3)
# print(subseq_ind.index)
approximate_match_subsequence_kmers(p,t)

[0, 1, 2, 14, 15, 16]

In [ ]:
def approximate_match_subsequence_kmers(p, t):
    matches = []
    chr_index = SubseqIndex(t,8,3)
    for i in range(0,len(p)+1):
        matches.extend(chr_index.query(p[i:]))
    return list(set(matches))
print(len(approximate_match_subsequence_kmers('GGCGCGGTGGCTCACGCCTGTAAT',chr1)))